<a href="https://colab.research.google.com/github/venuv/langchain_semantic_search/blob/main/langchain_semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install openai
!pip install requests transformers faiss-cpu
!pip install PyPDF2

  Using cached transformers-4.27.4-py3-none-any.whl (6.8 MB)
  Using cached huggingface_hub-0.13.4-py3-none-any.whl (200 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 33.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00a 0:00:01


In [3]:
from langchain.llms import OpenAI
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.docstore.document import Document
import requests
import os
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.text_splitter import CharacterTextSplitter
import pickle
import os


In [ ]:
%env OPENAI_API_KEY = <your key here. NO QUOTES>

env: OPENAI_API_KEY=<your key here. NO QUOTES>


In [ ]:
drive.mount('/content/drive')
gdrive_path = '/content/drive/MyDrive/sleep_pdf/' # or whatever your Google Folder path is

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Langchain Document - https://github.com/hwchase17/langchain/blob/master/langchain/docstore/document.py
def get_pdf_data(file_path, num_pages = 1):
  reader = PdfReader(gdrive_path+file_path)
  full_doc_text = ""
  for page in range(len(reader.pages)):
    current_page = reader.pages[page]
    text = current_page.extract_text()
    full_doc_text += text


  return Document(
        page_content=full_doc_text,
        metadata = {"source": file_path} 
    )

In [ ]:
def source_docs():
    return [get_pdf_data(file) for file in os.listdir(gdrive_path)]

In [ ]:
def search_index(source_docs):
    source_chunks = []
    splitter = CharacterTextSplitter(separator=" ", chunk_size=1024, chunk_overlap=0)

    for source in source_docs:
        for chunk in splitter.split_text(source.page_content):
            source_chunks.append(Document(page_content=chunk, metadata=source.metadata))
    #more likely you'll pickle this inde
    with open("search_index.pickle", "wb") as f:
        pickle.dump(FAISS.from_documents(source_chunks, OpenAIEmbeddings()), f)

In [ ]:
#faster but more expensive in $ - chain = load_qa_with_sources_chain(OpenAI(temperature=0),verbose=False,chain_type="map_reduce")
chain = load_qa_with_sources_chain(OpenAI(temperature=0),verbose=False, chain_type="stuff")
def print_answer(question):
    with open("search_index.pickle", "rb") as f:
        search_index = pickle.load(f)
        #print("type - ", type(search_index))
    print(
        chain(
            {
                "input_documents": search_index.similarity_search(question, k=3),
                "question": question,
            },
            return_only_outputs=True,
        )["output_text"]
    
    )
    

In [ ]:
import pdb
sources = source_docs()
search_index(sources)
try:
  print_answer(<your question on the document corpus here>) # your question here
except Exception as e:
  print(f"Unexpected error: {e}")

 Sleep devices and actigraphy can both be used to assess sleep quality. Sleep devices measure sleep quality on an accelerometer basis, while actigraphy measures sleep/vigilance patterns.
SOURCES: sleep device vs actigraphy.pdf, teeth grinding sleep quality.pdf
